In [1]:
import os
from dotenv import load_dotenv
import re

from groq import Groq

## Get the API key

In [2]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(len(GROQ_API_KEY))

56


## Client

In [3]:
client = Groq(
    api_key=GROQ_API_KEY,
)

## Prompts

In [4]:
# CLASS_PROMPT : classifier prompt
# NEWS_PROMPT : current events prompt
# COMPANY_PROMPT : company prompt
# THINK_PROMPT : thinking companion prompt

In [5]:
CLASS_PROMPT = """
Look at the user query.
Is it a question about a current event / political topic? If so, return 1.
Is it a question about a company or an investment opportunity? If so, return 2.
Is it a general question requiring a thinking companion? If so, return 3.
If it's impossible to fit any of the preceding descriptions, return 4.
Don't return anything else. Just one number. Nothing else. No explanations. You should return one single symbol. Double-check that.
"""

In [6]:
NEWS_PROMPT = """
You are a talented political commentator who's able to comment on a political event from multiple angles. 
You will be given a current political event/topic. 
Return a JSON object with the following data (look at the name of the parameter and its explanation): 
    "topic", // The name of the topic/event
    "timeline", // Date, like January 2020 -- March 2022
    "summary",  // This is a compressed UNBIASED summary of the commentaries from different political views
    "left_wing_commentary", // What is the left-wing view of the event?
    "right_wing_commentary", // What is the right-wing view of the event?
    "list_of_last_news". // Every piece of news is structured as follows: "news_story", "date", "source", "bias_level". "bias_level" is a float between 0.0 (unbiased) and 1.0 (extremely biased and one-sided).
Make sure that you respect the provided schema of the JSON object.
Don't return anything else. Just the JSON.
    """

In [12]:
COMPANY_PROMPT = """
You are a talented investor.
Provide fundamental analysis for a company.
Give the main information about its sector. Is it one of the top companies in its given field?
State its strong and weak points as an investment opportunity.
Estimate its ability to generate revenue.
Present some historical data about the stock price and the company's performance.
"""

## Classifier

In [7]:
def get_mode(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": CLASS_PROMPT,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    model_output = chat_completion.choices[0].message.content

    # Make sure that the classifier outputs a single number
    regex = re.compile(r'(\d+).*', flags=re.DOTALL)
    result = regex.search(model_output).group(1)
    result = int(result) if result is not None else 4

    return result

In [8]:
print(get_mode("Ukraine War"))
print(get_mode("Tesla, Inc."))
print(get_mode("Let's discuss Daniel Kahneman's book, Thinking Fast and Slow."))
print(get_mode("whatever whatever whatever whatever"))

1
2
3
4


## Quering

In [9]:
def news_query(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": NEWS_PROMPT,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content

In [10]:
print(news_query("Ukraine war"))

{
  "topic": "Ukraine war",
  "timeline": "February 2022 - present",
  "summary": "Russia's invasion of Ukraine has led to widespread condemnation from the international community and a humanitarian crisis. The conflict has resulted in thousands of deaths and displacement of millions of people.",
  "left_wing_commentary": "The left-wing view of the Ukraine war heavily criticizes Russia's actions as an unjustified invasion of a sovereign nation. They argue that Russian President Vladimir Putin's imperialist ambitions and aggressive foreign policy are to blame for the conflict. Additionally, they emphasize the need for diplomatic efforts and humanitarian aid for the victims of the war.",
  "right_wing_commentary": "The right-wing view of the Ukraine war is more varied, with some expressing support for Russia's actions and others condemning them. Those who support Russia argue that Western interference in Ukraine's affairs provoked the conflict and that Russia has legitimate security conc

In [ ]:
# add URL + critique by LLM

In [13]:
def company_query(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": COMPANY_PROMPT,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content

In [14]:
print(company_query("Tesla, Inc."))

Company Name: Tesla, Inc. (TSLA)

Sector: Tesla operates in the highly competitive and rapidly growing electric vehicle (EV) market, which is a subsector of the global automotive industry. The EV market is expected to grow significantly in the coming years due to increasing environmental awareness, government incentives, and advancements in battery technology.

Top Company: Yes, Tesla is one of the top companies in the EV market. It is the world's leading EV manufacturer by sales and has a strong brand image in the sector.

Strong Points:

1. Market Leader: Tesla is the market leader in the EV sector and has a first-mover advantage. It has a strong brand image, customer loyalty, and an expanding lineup of EVs.
2. Innovation: Tesla is known for its innovation and is a leader in battery technology, autonomous driving, and charging infrastructure.
3. Global Expansion: Tesla has a global presence with manufacturing facilities in the United States, China, and Europe. This allows the company